In [1]:
import gensim
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities
from nltk.tokenize import WordPunctTokenizer
import os
tk = WordPunctTokenizer() 

In [2]:
import shutil
shutil.rmtree('./brand_model', ignore_errors=True)

os.mkdir('./brand_model')

In [3]:
def preprocessing_text(input_text):
#     print(input_text)
    input_words_list = tk.tokenize(input_text)
#     print(input_words_list)
    clean_input_words_list = []
    for word in input_words_list:
        word = word.lower()
        result = list(filter(lambda x: x.isalpha(), word))
        word = ''.join(result)
#         print(result)
#         print(word)
        if word not in clean_input_words_list and len(word) > 2 :
            clean_input_words_list.append(word)
#     print(clean_input_words_list)
    return clean_input_words_list
            

In [4]:
def create_index(text_corpus) :

    frequency = defaultdict(int)
    for text in text_corpus:
        for token in text:
            frequency[token] += 1

    # Only keep words that appear more than once
    processed_corpus = [[token for token in text if frequency[token] > 1] for text in text_corpus]
    
    # creating dictonary from corpus
    dictionary = corpora.Dictionary(processed_corpus)
    
    # creating bow
    bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

    # train the model
    tfidf = models.TfidfModel(bow_corpus)

    index = similarities.SparseMatrixSimilarity(tfidf[bow_corpus], num_features=len(bow_corpus), num_best=10)
    return index, dictionary, tfidf

In [5]:
mapping_dict = {
"acuvue":"All Other GFO Brand",
"aveeno2":"Aveeno",
"bandaid":"Band Aid",
"benylin":"Benylin",
"helpthemquit":"Nicorette",
"lispro":"Listerine",
"lpm":"Le Petit Marseillais",
"motrin":"Motrin",
"nicoderm":"Nicoderm",
"pepcid":"Pepcid",
"sudafed":"Sudafed",
"visine":"Visine / Vispring / Visclear",
"polysporin" : "Polysporin",
"neutrogena" : "Neutrogena",
"cnc" : "Clean & Clear",
"jbaby3" : "Johnsons Baby",
'imodium': 'Imodium',
'tylenol' : 'Tylenol',
'lis' : "Listerine",
'nicorette' : "Nicorette",
'benadryl' : 'Benadryl',
'reactine' : 'Reactine',
'penaten' : 'Penaten',
'microlax' : 'Microlax/Micralax',
'rogaine' : 'Rogaine / Regaine',
'healthyessentials' : 'all',
'canvasdemo' : 'all',
'aveenomd' : 'Aveeno',
'lactaid' : 'Lactaid',
'bengay' : 'Bengay',
'desitin' : 'Desitin',
'zyrtec' : 'Zyrtec',
'neosporin' : 'Neosporin',
'blink' : 'All Other Self Care Brand',
'rhinocort' : 'Rhinocort/ Pulmicort'
}

In [6]:
def brand_wise_filter(df, brand_name_raw):
    brand_name = mapping_dict.get(brand_name_raw, 'all')
    if brand_name != 'all':
        temp_df = df.loc[df['gcph_brand'] == brand_name]
    else:
        temp_df = df

    text_corpus_brand = temp_df['clean_brand_desc'].tolist()
    try :
#         print("LOADING INDEX FOR {}".format(brand_name))
        dictionary_brand = gensim.corpora.Dictionary.load('./brand_model/{}_dictionary_brand'.format(brand_name_raw))
        tfidf_brand = gensim.models.TfidfModel.load('./brand_model/{}_tfidf_brand'.format(brand_name_raw))
        index_brand = similarities.SparseMatrixSimilarity.load('./brand_model/{}_index_brand'.format(brand_name_raw))
#         print("Successfully loaded INDEX FOR {}".format(brand_name))

    except Exception as e:
        print("Creating INDEX FOR {}".format(brand_name))
        index_brand, dictionary_brand, tfidf_brand = create_index(text_corpus_brand)
        index_brand.save('./brand_model/{}_index_brand'.format(brand_name_raw))
        dictionary_brand.save('./brand_model/{}_dictionary_brand'.format(brand_name_raw))
        tfidf_brand.save('./brand_model/{}_tfidf_brand'.format(brand_name_raw))
    return dictionary_brand, tfidf_brand, index_brand, temp_df

In [7]:
df = pd.read_csv('ca_product_hierarchy.csv')
df.dropna(subset=['material_name'],axis=0, inplace=True)
df['clean_brand_desc'] = df['material_name'].apply(lambda x: preprocessing_text(x))


In [8]:
def get_similar_doc(query_document, tfidf, dictionary, index):

    query_bow = dictionary.doc2bow(query_document)
    sims = index[tfidf[query_bow]]
    return sims

In [9]:
df2 = pd.read_csv('tamr_formatted_output_ca_20200714.csv')
df2 = df2[df2.match_type=='no_match']
df2['brand_desc'] = df2['page_title']+' '+df2['brand']
df2.sort_values("brand", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')
df2.dropna(subset=['brand_desc'],axis=0, inplace=True)

In [ ]:
query_list = df2.brand_desc.tolist()
final_output = []
for index, item in df2.iterrows():
    query = item['brand_desc']
#     print("Query is ==== >", query)
    dictionary_brand, tfidf_brand, index_brand, temp_df = brand_wise_filter(df,item['brand'])
    query_document = preprocessing_text(query)
    sims_brand = get_similar_doc(query_document, tfidf_brand, dictionary_brand, index_brand)
#     print(sims_brand, type(sims_brand))
    if sims_brand == []:
        print("Query is ==== >", query)
#         print("*********************Need to check with all data not just brand data")
#         print("Brand name is === >>> ",item['brand'])
#         dictionary_brand, tfidf_brand, index_brand, temp_df = brand_wise_filter(df,'all')
#         sims_brand = get_similar_doc(query_document, tfidf_brand, dictionary_brand, index_brand)

#     sims_cat = get_similar_doc(query_document, tfidf_cat, dictionary_cat, index_cat)
    
#     for document_number, score in sorted(enumerate(sims_brand), key=lambda x: x[1][1], reverse=True)[:1]:

#         document_number = score[0]
#         hierarchy_dict = {}
#         print("Match in Brand Sheet is : ",temp_df.iloc[document_number]['material_name'], "Score is : ", score[1])
# print("ALLL DONE")

Creating INDEX FOR All Other GFO Brand
Query is ==== > ACUVUEMD 2 à remplacement aux 2 semaines acuvue
Query is ==== > 1-DAY ACUVUE acuvue
Query is ==== > ACUVUE MOIST MULTIFOCAL (1-Day) Contact Lenses acuvue
Query is ==== > ACUVUE 2 2-Week Contact Lenses acuvue
Query is ==== > 1-DAY ACUVUE TruEye  acuvue
Query is ==== > ACUVUEMD VITAMD pour l'ASTIGMATISME acuvue
Query is ==== > ACUVUE OASYSMD multifocales pour la PRESBYTIE à remplacement aux 2 semaines acuvue
Query is ==== > ACUVUE OASYS for ASTIGMATISM 2-Week Contact Lenses acuvue
Query is ==== > ACUVUE OASYSMD 1-jour avec TECHNOLOGIE HydraLuxeMC pour l'ASTIGMATISME acuvue
Query is ==== > ACUVUE MOIST for ASTIGMATISM (1-Day) Contact Lenses acuvue
Query is ==== > ACUVUE VITA for ASTIGMATISM acuvue
Query is ==== > ACUVUE OASYS with Transitions Light Intelligent Technology acuvue
Query is ==== > ACUVUEMD MOIST (1-Day) acuvue
Query is ==== > 1-DAY ACUVUE DEFINE Contact Lenses acuvue
Query is ==== > OASYS 1-DAY with HydraLuxe acuvue
Query

In [ ]:
query = '  BENYLIN benylin'
print("Query is ==== >", query)
dictionary_brand, tfidf_brand, index_brand, temp_df = brand_wise_filter(df,'benylin')
query_document = preprocessing_text(query)
sims_brand = get_similar_doc(query_document, tfidf_brand, dictionary_brand, index_brand)
print(sims_brand)
#     sims_cat = get_similar_doc(query_document, tfidf_cat, dictionary_cat, index_cat)

for document_number, score in sorted(enumerate(sims_brand), key=lambda x: x[1][1], reverse=True)[:1]:

    document_number = score[0]
    hierarchy_dict = {}
    print("Match in Brand Sheet is : ",temp_df.iloc[document_number]['material_name'], "Score is : ", score[1])